<a id="section-one"></a>
<div style="padding:20px; 
            color:#150d0a;
            margin:10px;
            font-size:220%;
            text-align:center;
            display:fill;
            border-radius:20px;
            border-width: 5px;
            border-style: solid;
            border-color: #150d0a;
            background-color:#63F3B9;
            overflow:hidden;
            font-weight:500">Introduction</div>

## Summary and Table of Contents

This notebook contains traffic prediction EDA and a stacked (linear regression + XGBoost) prediction model tuned using Optuna. 

* [Introduction](#section-one)
* [Data Import and Splitting](#section-two)
* [EDA + Feature Engineering](#section-three)
* [Model Development](#section-four)
* [Model Testing](#section-five)
* [Results (Visualizations)](#section-six)
* [Conclusions and Potential Improvements](#section-seven)

## Set-up

In [1]:
# Installs
!pip install polars
!pip install lets-plot

# Imports
import polars as pl
import xgboost as xgb
import numpy as np
import optuna
import math
import statistics as stat

from lets_plot import *
from lets_plot.mapping import as_discrete
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

LetsPlot.setup_html()

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 4.0 MB/s eta 0:00:00


<a id="section-two"></a>
<div style="padding:20px; 
            color:#150d0a;
            margin:10px;
            font-size:220%;
            text-align:center;
            display:fill;
            border-radius:20px;
            border-width: 5px;
            border-style: solid;
            border-color: #150d0a;
            background-color:#63F3B9;
            overflow:hidden;
            font-weight:500">Data Import and Splitting</div>

## Importing the Data

The data contain an ID column that I dropped. Note that the data also contain a DateTime variable which polars can automatically read in the correct format. I also added a time index variable. The data contain counts of the number of vehicles at four traffic junctions.

In [2]:
# Importing the data as a polars data frame (dropping the ID column)
df = pl.read_csv("../input/traffic-prediction-dataset/traffic.csv", parse_dates = True).drop("ID")
df = df.with_row_count(name = "Time", offset = 0)
df.tail()

Time,DateTime,Junction,Vehicles
u32,datetime[μs],i64,i64
48115,2017-06-30 19:00:00,4,11
48116,2017-06-30 20:00:00,4,30
48117,2017-06-30 21:00:00,4,16
48118,2017-06-30 22:00:00,4,22
48119,2017-06-30 23:00:00,4,12


## Splitting 

I separated the full data set into four subsets: one for each junction. Thereafter, I split each subset into training and validation sets such that the validation set was all of June 2017 - the last 30 days of vehicle traffic in the data.

In [3]:
# Creating data sets for each junction
df_j1 = df.filter(pl.col("Junction") == 1)
df_j2 = df.filter(pl.col("Junction") == 2)
df_j3 = df.filter(pl.col("Junction") == 3)
df_j4 = df.filter(pl.col("Junction") == 4)

# Helper function
def df_splitter(df):
    """
    Splits a df into training and validation sets
       - Training: Before 2017-06-01 (YYYYMMDD)
       - Validation: 2017-06-01 - end of the data
       
    Assumes input is a polars df
    
    """
    
    df_train = df.filter(pl.col("DateTime") < pl.datetime(2017, 6, 1))
    df_valid = df.filter(pl.col("DateTime") >= pl.datetime(2017, 6, 1))
    
    return df_train, df_valid
    
# Splitting the data for each junction
df_j1_train, df_j1_valid = df_splitter(df_j1)
df_j2_train, df_j2_valid = df_splitter(df_j2)
df_j3_train, df_j3_valid = df_splitter(df_j3)
df_j4_train, df_j4_valid = df_splitter(df_j4)

<a id="section-three"></a>
<div style="padding:20px; 
            color:#150d0a;
            margin:10px;
            font-size:220%;
            text-align:center;
            display:fill;
            border-radius:20px;
            border-width: 5px;
            border-style: solid;
            border-color: #150d0a;
            background-color:#63F3B9;
            overflow:hidden;
            font-weight:500">EDA and Feature Engineering</div>

## Training Set EDA

First I simply plotted the time series for each junction. There are a few key takeaways from these plots:
1. Traffic increases over time for junctions 1 and 2
2. The junctions have non-constant variance across time 
3. Vehicle traffic is stable in the long run for junctions 3 and 4
4. Junction 4's data begins in 2017, whereas the rest begin in 2015

In [4]:
# Initializing colors for the remainder of the EDA
j1_color = '#F5AF0C'
j2_color = '#AE4FC6'
j3_color = '#76CDF2'
j4_color = '#76F2B8'

# Time series plot for each junction's training set
plt_ts_j1 = \
    ggplot(df_j1_train)+\
    geom_line(aes(x = "DateTime", y = "Vehicles"), 
              color = j1_color, sampling = "none")+\
    scale_x_datetime(format = "%b %Y")+\
    theme(plot_title = element_text(hjust = 0.5))+\
    flavor_darcula()+\
    labs(x = "Date", y = "Vehicles", title = "Junction 1")

plt_ts_j2 = \
    ggplot(df_j2_train)+\
    geom_line(aes(x = "DateTime", y = "Vehicles"), 
              color = j2_color, sampling = "none")+\
    scale_x_datetime(format = "%b %Y")+\
    theme(plot_title = element_text(hjust = 0.5))+\
    flavor_darcula()+\
    labs(x = "Date", y = "Vehicles", title = "Junction 2")

plt_ts_j3 = \
    ggplot(df_j3_train)+\
    geom_line(aes(x = "DateTime", y = "Vehicles"), 
              color = j3_color, sampling = "none")+\
    scale_x_datetime(format = "%b %Y")+\
    theme(plot_title = element_text(hjust = 0.5))+\
    flavor_darcula()+\
    labs(x = "Date", y = "Vehicles", title = "Junction 3")

plt_ts_j4 = \
    ggplot(df_j4_train)+\
    geom_line(aes(x = "DateTime", y = "Vehicles"), 
              color = j4_color, sampling = "none")+\
    scale_x_datetime(format = "%b %Y")+\
    theme(plot_title = element_text(hjust = 0.5))+\
    flavor_darcula()+\
    labs(x = "Date", y = "Vehicles", title = "Junction 4")

# Display each plot on a 4x1 grid
ts_plts = GGBunch()
ts_plts.add_plot(plt_ts_j1, 0, 0, 850, 300)
ts_plts.add_plot(plt_ts_j2, 0, 320, 850, 300)
ts_plts.add_plot(plt_ts_j3, 0, 640, 850, 300)
ts_plts.add_plot(plt_ts_j4, 0, 960, 850, 300)
ts_plts

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## Creating New Features

Below I created indicator features in each training set for:
- The year
- Month
- Date (within a month)
- Day of the week
- Hour

In [5]:
# Year features
df_j1_train = df_j1_train.with_columns(pl.col("DateTime").dt.year().alias("Year"))
df_j2_train = df_j2_train.with_columns(pl.col("DateTime").dt.year().alias("Year"))
df_j3_train = df_j3_train.with_columns(pl.col("DateTime").dt.year().alias("Year"))
df_j4_train = df_j4_train.with_columns(pl.col("DateTime").dt.year().alias("Year"))

# Month features
df_j1_train = df_j1_train.with_columns(pl.col("DateTime").dt.month().alias("Month"))
df_j2_train = df_j2_train.with_columns(pl.col("DateTime").dt.month().alias("Month"))
df_j3_train = df_j3_train.with_columns(pl.col("DateTime").dt.month().alias("Month"))
df_j4_train = df_j4_train.with_columns(pl.col("DateTime").dt.month().alias("Month"))

# Date within a month features
df_j1_train = df_j1_train.with_columns(pl.col("DateTime").dt.day().alias("Day_month"))
df_j2_train = df_j2_train.with_columns(pl.col("DateTime").dt.day().alias("Day_month"))
df_j3_train = df_j3_train.with_columns(pl.col("DateTime").dt.day().alias("Day_month"))
df_j4_train = df_j4_train.with_columns(pl.col("DateTime").dt.day().alias("Day_month"))

# Date within a week features
df_j1_train = df_j1_train.with_columns(pl.col("DateTime").dt.weekday().alias("Day_week"))
df_j2_train = df_j2_train.with_columns(pl.col("DateTime").dt.weekday().alias("Day_week"))
df_j3_train = df_j3_train.with_columns(pl.col("DateTime").dt.weekday().alias("Day_week"))
df_j4_train = df_j4_train.with_columns(pl.col("DateTime").dt.weekday().alias("Day_week"))

# Hour
df_j1_train = df_j1_train.with_columns(pl.col("DateTime").dt.hour().alias("Hour"))
df_j2_train = df_j2_train.with_columns(pl.col("DateTime").dt.hour().alias("Hour"))
df_j3_train = df_j3_train.with_columns(pl.col("DateTime").dt.hour().alias("Hour"))
df_j4_train = df_j4_train.with_columns(pl.col("DateTime").dt.hour().alias("Hour"))

### Traffic trends across different timeframes

The plots below are based on the newly derived features. From these plots I observed the following:
1. Most junctions experience less traffic on the weekends
2. Traffic hits a minimum around 5am for all junctions
3. Average traffic by day of the month is fairly stable
4. There is <6 months of traffic data for junction 4

In [6]:
# Helper function (Polars expression)
def mean_vehicles(df) -> pl.Expr:
    return pl.col("Vehicles").mean()

# Combine all of the training sets to make plotting easier
df_train = pl.concat([df_j1_train, df_j2_train, df_j3_train, df_j4_train])

# Group by junction and month, average vehicles 
df_monthly = (
    df_train.groupby(["Junction", "Month"])
    .agg([mean_vehicles("Month")])
    .sort("Junction")
)

# Group by junction and day of week, average vehicles
df_day_week = (
    df_train.groupby(["Junction", "Day_week"])
    .agg([mean_vehicles("Day_week")])
    .sort("Junction")
)

# Group by junction and day of month, average vehicles
df_day_month = (
    df_train.groupby(["Junction", "Day_month"])
    .agg([mean_vehicles("Day_month")])
    .sort("Junction")
)

# Group by junction and hour, average vehicles
df_hourly = (
    df_train.groupby(["Junction", "Hour"])
    .agg([mean_vehicles("Hour")])
    .sort("Junction")
)

# Monthly plot
plt_monthly = \
    ggplot(df_monthly)+\
    geom_line(aes(x = "Month", y = "Vehicles", color = as_discrete("Junction")), size = 3)+\
    scale_x_discrete(breaks = list(range(1,13,1)))+\
    scale_color_manual(values = [j1_color, j2_color, j3_color, j4_color])+\
    theme(plot_title = element_text(hjust = 0.5))+\
    flavor_darcula()+\
    labs(x = "Month", y = "Vehicles", title = "Average Monthly Traffic")

# Day of week plot
plt_day_week = \
    ggplot(df_day_week)+\
    geom_line(aes(x = "Day_week", y = "Vehicles", color = as_discrete("Junction")), size = 3)+\
    scale_x_discrete(breaks = list(range(1,8,1)))+\
    scale_color_manual(values = [j1_color, j2_color, j3_color, j4_color])+\
    theme(plot_title = element_text(hjust = 0.5))+\
    flavor_darcula()+\
    labs(x = "Day of the Week", y = "Vehicles", title = "Average Traffic by Day of the Week")


# Day of month plot
plt_day_month = \
    ggplot(df_day_month)+\
    geom_line(aes(x = "Day_month", y = "Vehicles", color = as_discrete("Junction")), size = 3)+\
    scale_x_discrete(breaks = list(range(1,32,1)))+\
    scale_color_manual(values = [j1_color, j2_color, j3_color, j4_color])+\
    theme(plot_title = element_text(hjust = 0.5))+\
    flavor_darcula()+\
    labs(x = "Day of the Month", y = "Vehicles", title = "Average Traffic by Day of the Month")

# Hourly Plot
plt_hourly = \
    ggplot(df_hourly)+\
    geom_line(aes(x = "Hour", y = "Vehicles", color = as_discrete("Junction")), size = 3)+\
    scale_color_manual(values = [j1_color, j2_color, j3_color, j4_color])+\
    theme(plot_title = element_text(hjust = 0.5))+\
    flavor_darcula()+\
    labs(x = "Hour", y = "Vehicles", title = "Average Hourly Traffic")

# Display each plot on a 2x2 grid
ts_plts_2 = GGBunch()
ts_plts_2.add_plot(plt_monthly, 0, 0, 700, 300)
ts_plts_2.add_plot(plt_day_week, 0, 320, 700, 300)
ts_plts_2.add_plot(plt_day_month, 0, 640, 700, 300)
ts_plts_2.add_plot(plt_hourly, 0, 960, 700, 300)
ts_plts_2

## More Feature Engineering

Given the insights gained from the plots above I decided to create the following features:
- Weekend indicator for Junctions 1 and 2
- Midnight to 5am indicator for all junctions
- 5am to noon indicator for all junctions
- Noon to midnight indicator for all junctions

In [7]:
# Weekend indicator for junction 1
df_j1_train = df_j1_train.with_columns([
    (pl.when(pl.col("Day_week") == 7)
    .then(1)
    .when(pl.col("Day_week") == 6)
    .then(1)
    .otherwise(0))
    .alias("Weekend")
])

# Weekend indicator for junction 2
df_j2_train = df_j2_train.with_columns([
    (pl.when(pl.col("Day_week") == 7)
    .then(1)
    .when(pl.col("Day_week") == 6)
    .then(1)
    .otherwise(0))
    .alias("Weekend")
])

# Midnight to 5am indicator for all junctions
df_j1_train = df_j1_train.with_columns([
    (pl.when((pl.col("Hour") >= 0) & (pl.col("Hour") <= 5))
     .then(1)
     .otherwise(0))
     .alias("Mid_to_five")
])

df_j2_train = df_j2_train.with_columns([
    (pl.when((pl.col("Hour") >= 0) & (pl.col("Hour") <= 5))
     .then(1)
     .otherwise(0))
     .alias("Mid_to_five")
])

df_j3_train = df_j3_train.with_columns([
    (pl.when((pl.col("Hour") >= 0) & (pl.col("Hour") <= 5))
     .then(1)
     .otherwise(0))
     .alias("Mid_to_five")
])

df_j4_train = df_j4_train.with_columns([
    (pl.when((pl.col("Hour") >= 0) & (pl.col("Hour") <= 5))
     .then(1)
     .otherwise(0))
     .alias("Mid_to_five")
])

# 5am to noon indicator for all junctions
df_j1_train = df_j1_train.with_columns([
    (pl.when((pl.col("Hour") >= 5) & (pl.col("Hour") <= 12))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])

df_j2_train = df_j2_train.with_columns([
    (pl.when((pl.col("Hour") >= 5) & (pl.col("Hour") <= 12))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])

df_j3_train = df_j3_train.with_columns([
    (pl.when((pl.col("Hour") >= 5) & (pl.col("Hour") <= 12))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])

df_j4_train = df_j4_train.with_columns([
    (pl.when((pl.col("Hour") >= 5) & (pl.col("Hour") <= 12))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])


# Noon to midnight indicator
df_j1_train = df_j1_train.with_columns([
    (pl.when((pl.col("Hour") >= 12) & (pl.col("Hour") <= 0))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])

df_j2_train = df_j2_train.with_columns([
    (pl.when((pl.col("Hour") >= 12) & (pl.col("Hour") <= 0))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])

df_j3_train = df_j3_train.with_columns([
    (pl.when((pl.col("Hour") >= 12) & (pl.col("Hour") <= 0))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])

df_j4_train = df_j4_train.with_columns([
    (pl.when((pl.col("Hour") >= 12) & (pl.col("Hour") <= 0))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])

## Transforming the Training Data

As mentioned previously, many of the junctions have non-constant variance across time. As shown in the plots below, the log transform smoothed out the varaince in each junction. However, the time series for junctions 3 and 4 are still fairly messy looking, but I decided not to manipulate the training data any further as the model I used doesn't require the series to be stationary.

In [8]:
# Junction 1 log transform
df_j1_train = df_j1_train.with_columns([
    (pl.col("Vehicles").log()).alias("Vehicles_log")
])

# Junction 2 log transform
df_j2_train = df_j2_train.with_columns([
    (pl.col("Vehicles").log()).alias("Vehicles_log")
])

# Junction 3 log transform
df_j3_train = df_j3_train.with_columns([
    (pl.col("Vehicles").log()).alias("Vehicles_log")
])

# Junction 4 log transform
df_j4_train = df_j4_train.with_columns([
    (pl.col("Vehicles").log()).alias("Vehicles_log")
])

In [9]:
# Plotting the results before and after
# Junction 1
plt_ts_j1_log = \
    ggplot(df_j1_train)+\
    geom_line(aes(x = "DateTime", y = "Vehicles_log"), 
              color = j1_color, sampling = "none")+\
    scale_x_datetime(format = "%b %Y")+\
    theme(plot_title = element_text(hjust = 0.5))+\
    flavor_darcula()+\
    labs(x = "Date", y = "Log Vehicles", title = "Junction 1 (After Log Transform)")

# Junction 2
plt_ts_j2_log = \
    ggplot(df_j2_train)+\
    geom_line(aes(x = "DateTime", y = "Vehicles_log"), 
              color = j2_color, sampling = "none")+\
    scale_x_datetime(format = "%b %Y")+\
    theme(plot_title = element_text(hjust = 0.5))+\
    flavor_darcula()+\
    labs(x = "Date", y = "Log Vehicles", title = "Junction 2 (After Log Transform)")

# Junction 3
plt_ts_j3_log = \
    ggplot(df_j3_train)+\
    geom_line(aes(x = "DateTime", y = "Vehicles_log"), 
              color = j3_color, sampling = "none")+\
    scale_x_datetime(format = "%b %Y")+\
    theme(plot_title = element_text(hjust = 0.5))+\
    flavor_darcula()+\
    labs(x = "Date", y = "Log Vehicles", title = "Junction 3 (After Log Transform)")

# Junction 4
plt_ts_j4_log = \
    ggplot(df_j4_train)+\
    geom_line(aes(x = "DateTime", y = "Vehicles_log"), 
              color = j4_color, sampling = "none")+\
    scale_x_datetime(format = "%b %Y")+\
    theme(plot_title = element_text(hjust = 0.5))+\
    flavor_darcula()+\
    labs(x = "Date", y = "Log Vehicles", title = "Junction 4 (After Log Transform)")


ts_plts_log = GGBunch()
ts_plts_log.add_plot(plt_ts_j1_log, 0, 0, 500, 300)
ts_plts_log.add_plot(plt_ts_j1, 500, 0, 500, 300)
ts_plts_log.add_plot(plt_ts_j2_log, 0, 320, 500, 300)
ts_plts_log.add_plot(plt_ts_j2, 500, 320, 500, 300)
ts_plts_log.add_plot(plt_ts_j3_log, 0, 640, 500, 300)
ts_plts_log.add_plot(plt_ts_j3, 500, 640, 500, 300)
ts_plts_log.add_plot(plt_ts_j4_log, 0, 960, 500, 300)
ts_plts_log.add_plot(plt_ts_j4, 500, 960, 500, 300)
ts_plts_log

## Pre-Processing the Validation Data

Below I pre-process the validation data in the exact same way as the training data. 

In [10]:
# Year features
df_j1_valid = df_j1_valid.with_columns(pl.col("DateTime").dt.year().alias("Year"))
df_j2_valid = df_j2_valid.with_columns(pl.col("DateTime").dt.year().alias("Year"))
df_j3_valid = df_j3_valid.with_columns(pl.col("DateTime").dt.year().alias("Year"))
df_j4_valid = df_j4_valid.with_columns(pl.col("DateTime").dt.year().alias("Year"))

# Month features
df_j1_valid = df_j1_valid.with_columns(pl.col("DateTime").dt.month().alias("Month"))
df_j2_valid = df_j2_valid.with_columns(pl.col("DateTime").dt.month().alias("Month"))
df_j3_valid = df_j3_valid.with_columns(pl.col("DateTime").dt.month().alias("Month"))
df_j4_valid = df_j4_valid.with_columns(pl.col("DateTime").dt.month().alias("Month"))

# Date within a month features
df_j1_valid = df_j1_valid.with_columns(pl.col("DateTime").dt.day().alias("Day_month"))
df_j2_valid = df_j2_valid.with_columns(pl.col("DateTime").dt.day().alias("Day_month"))
df_j3_valid = df_j3_valid.with_columns(pl.col("DateTime").dt.day().alias("Day_month"))
df_j4_valid = df_j4_valid.with_columns(pl.col("DateTime").dt.day().alias("Day_month"))

# Date within a week features
df_j1_valid = df_j1_valid.with_columns(pl.col("DateTime").dt.weekday().alias("Day_week"))
df_j2_valid = df_j2_valid.with_columns(pl.col("DateTime").dt.weekday().alias("Day_week"))
df_j3_valid = df_j3_valid.with_columns(pl.col("DateTime").dt.weekday().alias("Day_week"))
df_j4_valid = df_j4_valid.with_columns(pl.col("DateTime").dt.weekday().alias("Day_week"))

# Hour
df_j1_valid = df_j1_valid.with_columns(pl.col("DateTime").dt.hour().alias("Hour"))
df_j2_valid = df_j2_valid.with_columns(pl.col("DateTime").dt.hour().alias("Hour"))
df_j3_valid = df_j3_valid.with_columns(pl.col("DateTime").dt.hour().alias("Hour"))
df_j4_valid = df_j4_valid.with_columns(pl.col("DateTime").dt.hour().alias("Hour"))

# Weekend indicator for junction 1
df_j1_valid = df_j1_valid.with_columns([
    (pl.when(pl.col("Day_week") == 7)
    .then(1)
    .when(pl.col("Day_week") == 6)
    .then(1)
    .otherwise(0))
    .alias("Weekend")
])

# Weekend indicator for junction 2
df_j2_valid = df_j2_valid.with_columns([
    (pl.when(pl.col("Day_week") == 7)
    .then(1)
    .when(pl.col("Day_week") == 6)
    .then(1)
    .otherwise(0))
    .alias("Weekend")
])

# Midnight to 5am indicator for all junctions
df_j1_valid = df_j1_valid.with_columns([
    (pl.when((pl.col("Hour") >= 0) & (pl.col("Hour") <= 5))
     .then(1)
     .otherwise(0))
     .alias("Mid_to_five")
])

df_j2_valid = df_j2_valid.with_columns([
    (pl.when((pl.col("Hour") >= 0) & (pl.col("Hour") <= 5))
     .then(1)
     .otherwise(0))
     .alias("Mid_to_five")
])

df_j3_valid = df_j3_valid.with_columns([
    (pl.when((pl.col("Hour") >= 0) & (pl.col("Hour") <= 5))
     .then(1)
     .otherwise(0))
     .alias("Mid_to_five")
])

df_j4_valid = df_j4_valid.with_columns([
    (pl.when((pl.col("Hour") >= 0) & (pl.col("Hour") <= 5))
     .then(1)
     .otherwise(0))
     .alias("Mid_to_five")
])

# 5am to noon indicator for all junctions
df_j1_valid = df_j1_valid.with_columns([
    (pl.when((pl.col("Hour") >= 5) & (pl.col("Hour") <= 12))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])

df_j2_valid = df_j2_valid.with_columns([
    (pl.when((pl.col("Hour") >= 5) & (pl.col("Hour") <= 12))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])

df_j3_valid = df_j3_valid.with_columns([
    (pl.when((pl.col("Hour") >= 5) & (pl.col("Hour") <= 12))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])

df_j4_valid = df_j4_valid.with_columns([
    (pl.when((pl.col("Hour") >= 5) & (pl.col("Hour") <= 12))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])


# Noon to midnight indicator
df_j1_valid = df_j1_valid.with_columns([
    (pl.when((pl.col("Hour") >= 12) & (pl.col("Hour") <= 0))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])

df_j2_valid = df_j2_valid.with_columns([
    (pl.when((pl.col("Hour") >= 12) & (pl.col("Hour") <= 0))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])

df_j3_valid = df_j3_valid.with_columns([
    (pl.when((pl.col("Hour") >= 12) & (pl.col("Hour") <= 0))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])

df_j4_valid = df_j4_valid.with_columns([
    (pl.when((pl.col("Hour") >= 12) & (pl.col("Hour") <= 0))
     .then(1)
     .otherwise(0))
     .alias("Five_to_noon")
])

# Junction 1 log transform
df_j1_valid = df_j1_valid.with_columns([
    (pl.col("Vehicles").log()).alias("Vehicles_log")
])

# Junction 2 log transform
df_j2_valid = df_j2_valid.with_columns([
    (pl.col("Vehicles").log()).alias("Vehicles_log")
])

# Junction 3 log transform
df_j3_valid = df_j3_valid.with_columns([
    (pl.col("Vehicles").log()).alias("Vehicles_log")
])

# Junction 4 log transform
df_j4_valid = df_j4_valid.with_columns([
    (pl.col("Vehicles").log()).alias("Vehicles_log")
])

<a id="section-four"></a>
<div style="padding:20px; 
            color:#150d0a;
            margin:10px;
            font-size:220%;
            text-align:center;
            display:fill;
            border-radius:20px;
            border-width: 5px;
            border-style: solid;
            border-color: #150d0a;
            background-color:#63F3B9;
            overflow:hidden;
            font-weight:500">Model Development</div>

## Development Summary
1. Stacked model: linear regression first layer, XGB second layer
    - Linear regression make initial predictions
    - XGB predicts the residuals given by the regression
    - The predictions from both are summed to give the final forecast values
2. The XGB model hyperparameters were tuned using Optuna
3. The stacked model was trained using the log transformed target

**....** I'm not sure if this type of approach is technically "stacking" as the second models predicts the residuals of the first instead of using the first model's predictions as features. If anyone has a better name please **leave a comment.**

In [11]:
# Creating Optuna objective function
def objective(trial):
    
    """
    Optuna objective function, minimizes the
    RMSE for a stacked regression + XGB
    time series forecasting model
    
    """
    
    # Get data for the regression model
    xtrain = df_train.drop(["DateTime", "Junction", "Vehicles", "Vehicles_log"]).to_numpy()
    xvalid = df_valid.drop(["DateTime", "Junction", "Vehicles", "Vehicles_log"]).to_numpy()

    ytrain = df_train.get_column("Vehicles_log").to_numpy()
    yvalid = df_valid.get_column("Vehicles_log").to_numpy()
    
    # Regression model
    reg_model = LinearRegression().fit(xtrain, ytrain)
    
    # Regression predictions
    reg_preds_train = reg_model.predict(xtrain)
    reg_preds_valid = reg_model.predict(xvalid)
    
    # Regression residuals 
    reg_resid_train = (ytrain - reg_preds_train)
    reg_resid_valid = (yvalid - reg_preds_valid)
    
    # Suggest hyperparameters for XGBoost
    params = {'objective': 'reg:squarederror',
              'eval_metric': 'rmse',
              'seed': 19970507,
              'eta': trial.suggest_float("eta", 1e-2, 0.25, log = True),
              'max_depth': trial.suggest_int("max_depth", 1, 7),
              'lambda': trial.suggest_float("lambda", 1e-8, 100.0, log = True),
              'alpha': trial.suggest_float("alpha", 1e-8, 100.0, log = True),
             }
    
    # Setup the data for the XGB model
    dmat_train = xgb.DMatrix(xtrain, label = reg_resid_train)
    dmat_valid = xgb.DMatrix(xvalid, label = reg_resid_valid)
    
    watchlist = [(dmat_train, 'train'), (dmat_valid, 'eval')]
    
    # Fit and predict
    xgb_model = xgb.train(params, 
                          dtrain = dmat_train, 
                          num_boost_round = trial.suggest_int("num_boost_round", 20, 3000),
                          evals = watchlist,
                          verbose_eval = False)
    
    xgb_preds_valid = xgb_model.predict(dmat_valid) 
    
    # Sum predictions to get final preds 
    preds = (reg_preds_valid + xgb_preds_valid)
    
    # Return the RMSE
    return math.sqrt(mean_squared_error(yvalid, preds))

In [12]:
# Set-up
optuna.logging.set_verbosity(optuna.logging.WARNING) # Suppress log messages 

# Junction 1 Optuna study
df_train = df_j1_train
df_valid = df_j1_valid

study_j1 = optuna.create_study(direction = 'minimize')
study_j1.optimize(objective, n_trials = 5)

# Junction 2 Optuna study
df_train = df_j2_train
df_valid = df_j2_valid

study_j2 = optuna.create_study(direction = 'minimize')
study_j2.optimize(objective, n_trials = 5)

# Junction 3 Optuna study
df_train = df_j3_train
df_valid = df_j3_valid

study_j3 = optuna.create_study(direction = 'minimize')
study_j3.optimize(objective, n_trials = 5)

# Junction 4 Optuna study
df_train = df_j4_train
df_valid = df_j4_valid

study_j4 = optuna.create_study(direction = 'minimize')
study_j4.optimize(objective, n_trials = 5)

<a id="section-five"></a>
<div style="padding:20px; 
            color:#150d0a;
            margin:10px;
            font-size:220%;
            text-align:center;
            display:fill;
            border-radius:20px;
            border-width: 5px;
            border-style: solid;
            border-color: #150d0a;
            background-color:#63F3B9;
            overflow:hidden;
            font-weight:500">Model Testing</div>

## Fit and Predict

Below I take the best hyperparameters for each model, train the stacked model to each junction's training set, and make predictions on the validation sets. Note that the predictions were trasnformed back to the original scale to calculate the RMSE on the validation set. The RMSE results should be compared with caution as the range of values for each junction varied widely.

In [13]:
def final_model_preds(optuna_params, df_train, df_valid, junction):
    """
    This function takes the best optuna parameters and
    creates a final stacked regression + XGB model
    
    Returns:
     - Model predictions on the validation set
     - RMSE on the validation set
     
    NOTE: while the optuna function optimized the 
    hyperparameters with respect to log(Vehicles), this
    function returns the predictions and RMSE on the original scale
    
    """
    
    # Training and validation sets 
    xtrain = df_train.drop(["DateTime", "Junction", "Vehicles", "Vehicles_log"]).to_numpy()
    xvalid = df_valid.drop(["DateTime", "Junction", "Vehicles", "Vehicles_log"]).to_numpy()

    ytrain = df_train.get_column("Vehicles_log").to_numpy()
    yvalid = df_valid.get_column("Vehicles_log").to_numpy()
    
    yvalid_orig = df_valid.get_column("Vehicles").to_numpy()

    # Fit the regression model
    reg_model = LinearRegression().fit(xtrain, ytrain)
    
    # Regression predictions
    reg_preds_train = reg_model.predict(xtrain)
    reg_preds_valid = reg_model.predict(xvalid)
    
    # Regression residuals 
    reg_resid_train = (ytrain - reg_preds_train)
    reg_resid_valid = (yvalid - reg_preds_valid)
    
    # Get the best hyperparameters for the XGBoost model
    best_params = {'objective': 'reg:squarederror',
                  'eval_metric': 'rmse',
                  'seed': 19970507,
                  'eta': optuna_params['eta'],
                  'max_depth': optuna_params['max_depth'],
                  'lambda': optuna_params['lambda'],
                  'alpha': optuna_params['alpha'],
                 }

    # Set up the dence matrices for XGBoost
    dmat_train = xgb.DMatrix(xtrain, label = reg_resid_train)
    dmat_valid = xgb.DMatrix(xvalid, label = reg_resid_valid)
    
    watchlist = [(dmat_train, 'train'), (dmat_valid, 'eval')]

    # Train the XGBoost model
    xgb_model = xgb.train(best_params, 
                          dtrain = dmat_train, 
                          num_boost_round = optuna_params['num_boost_round'],
                          evals = watchlist,
                          early_stopping_rounds = 100, 
                          verbose_eval = False)
    
    xgb_preds_valid = xgb_model.predict(dmat_valid) 

    # Sum predictions to get final preds
    preds = (reg_preds_valid + xgb_preds_valid)
    
    # Get predictions on the original scale
    preds_orig = [math.exp(x) for x in preds]
    
    # Calculate RMSE
    rmse = math.sqrt(mean_squared_error(yvalid_orig, preds_orig))

    # Print RMSE
    print("Validation set RMSE for Junction", junction, ": ", rmse)
    
    # Return predictions and RMSE
    return preds_orig, rmse


In [14]:
# Junction 1 final predictions
preds_j1, valid_rmse_j1 = final_model_preds(optuna_params = study_j1.best_params, 
                                            df_train = df_j1_train, 
                                            df_valid = df_j1_valid,
                                            junction = 1)

# Junction 1 final predictions
preds_j2, valid_rmse_j2 = final_model_preds(optuna_params = study_j2.best_params, 
                                            df_train = df_j2_train, 
                                            df_valid = df_j2_valid,
                                            junction = 2)

# Junction 1 final predictions
preds_j3, valid_rmse_j3 = final_model_preds(optuna_params = study_j3.best_params, 
                                            df_train = df_j3_train, 
                                            df_valid = df_j3_valid,
                                            junction = 3)

# Junction 1 final predictions
preds_j4, valid_rmse_j4 = final_model_preds(optuna_params = study_j4.best_params, 
                                            df_train = df_j4_train, 
                                            df_valid = df_j4_valid,
                                            junction = 4)

Validation set RMSE for Junction 1 :  7.376030114861925
Validation set RMSE for Junction 2 :  3.41293724266455
Validation set RMSE for Junction 3 :  8.442189285199607
Validation set RMSE for Junction 4 :  3.641679231759858


<a id="section-six"></a>
<div style="padding:20px; 
            color:#150d0a;
            margin:10px;
            font-size:220%;
            text-align:center;
            display:fill;
            border-radius:20px;
            border-width: 5px;
            border-style: solid;
            border-color: #150d0a;
            background-color:#63F3B9;
            overflow:hidden;
            font-weight:500">Results (Visualizations)</div>

## Visualizing the Results

Below I plotted the true values from the validation sets against the predictions from the stacked models. From the plots it is evident that:
1. The models for junctions 1 and 2 were reasonably accurate
2. The model for junction 3 was horrible
3. The model for juntion 4 was moderatley good at predicting traffic flow

In [15]:
# Junction 1
df_j1_labels = pl.DataFrame(
    {'DateTime': df_j1_valid.get_column("DateTime"), 
     'Vehicles': df_j1_valid.get_column("Vehicles"), 
     'Group': ["Label"]*len(df_j1_valid)}
)

df_j1_preds = pl.DataFrame(
    {'DateTime_preds': df_j1_valid.get_column("DateTime"), 
     'Vehicles_preds': preds_j1, 
     'Group_preds': ["Predictions"]*len(df_j1_valid)}
)

df_j1 = (
    pl.concat([df_j1_labels, df_j1_preds], how = 'horizontal')
    .with_columns(
        (pl.lit("True Values").alias("Group_label")),
        (pl.lit("Predictions").alias("Group_pred")))
)

plt_j1 = \
    ggplot(df_j1)+\
    geom_line(aes(x = "DateTime", y = "Vehicles", color = "Group_label"), 
              sampling = "none", size = 0.5, show_legend = True)+\
    geom_line(aes(x = "DateTime", y = "Vehicles_preds", color = "Group_pred"), 
              sampling = "none", size = 0.5, show_legend = True)+\
    scale_color_manual(values = ['white', j1_color])+\
    scale_x_datetime(format = "%Y-%m-%d")+\
    scale_y_continuous(limits = [20, 145])+\
    theme(plot_title = element_text(hjust = 0.5),
         legend_title = element_blank())+\
    flavor_darcula()+\
    labs(x = "Date", y = "Vehicles", title = "Junction 1")

# Junction 2
df_j2_labels = pl.DataFrame(
    {'DateTime': df_j2_valid.get_column("DateTime"), 
     'Vehicles': df_j2_valid.get_column("Vehicles"), 
     'Group': ["Label"]*len(df_j2_valid)}
)

df_j2_preds = pl.DataFrame(
    {'DateTime_preds': df_j2_valid.get_column("DateTime"), 
     'Vehicles_preds': preds_j2, 
     'Group_preds': ["Predictions"]*len(df_j2_valid)}
)

df_j2 = (
    pl.concat([df_j2_labels, df_j2_preds], how = 'horizontal')
    .with_columns(
        (pl.lit("True Values").alias("Group_label")),
        (pl.lit("Predictions").alias("Group_pred")))
)

plt_j2 = \
    ggplot(df_j2)+\
    geom_line(aes(x = "DateTime", y = "Vehicles", color = "Group_label"), 
              sampling = "none", size = 0.5, show_legend = True)+\
    geom_line(aes(x = "DateTime", y = "Vehicles_preds", color = "Group_pred"), 
              sampling = "none", size = 0.5, show_legend = True)+\
    scale_color_manual(values = ['white', j2_color])+\
    scale_x_datetime(format = "%Y-%m-%d")+\
    scale_y_continuous(limits = [20, 50])+\
    theme(plot_title = element_text(hjust = 0.5),
         legend_title = element_blank())+\
    flavor_darcula()+\
    labs(x = "Date", y = "Vehicles", title = "Junction 2")

# Junction 3
df_j3_labels = pl.DataFrame(
    {'DateTime': df_j3_valid.get_column("DateTime"), 
     'Vehicles': df_j3_valid.get_column("Vehicles"), 
     'Group': ["Label"]*len(df_j3_valid)}
)

df_j3_preds = pl.DataFrame(
    {'DateTime_preds': df_j3_valid.get_column("DateTime"), 
     'Vehicles_preds': preds_j3, 
     'Group_preds': ["Predictions"]*len(df_j3_valid)}
)

df_j3 = (
    pl.concat([df_j3_labels, df_j3_preds], how = 'horizontal')
    .with_columns(
        (pl.lit("True Values").alias("Group_label")),
        (pl.lit("Predictions").alias("Group_pred")))
)

plt_j3 = \
    ggplot(df_j3)+\
    geom_line(aes(x = "DateTime", y = "Vehicles", color = "Group_label"), 
              sampling = "none", size = 0.5, show_legend = True)+\
    geom_line(aes(x = "DateTime", y = "Vehicles_preds", color = "Group_pred"), 
              sampling = "none", size = 0.5, show_legend = True)+\
    scale_color_manual(values = ['white', j3_color])+\
    scale_x_datetime(format = "%Y-%m-%d")+\
    scale_y_continuous(limits = [20, 120])+\
    theme(plot_title = element_text(hjust = 0.5),
         legend_title = element_blank())+\
    flavor_darcula()+\
    labs(x = "Date", y = "Vehicles", title = "Junction 3")

# Junction 4
df_j4_labels = pl.DataFrame(
    {'DateTime': df_j4_valid.get_column("DateTime"), 
     'Vehicles': df_j4_valid.get_column("Vehicles"), 
     'Group': ["Label"]*len(df_j4_valid)}
)

df_j4_preds = pl.DataFrame(
    {'DateTime_preds': df_j4_valid.get_column("DateTime"), 
     'Vehicles_preds': preds_j4, 
     'Group_preds': ["Predictions"]*len(df_j4_valid)}
)

df_j4 = (
    pl.concat([df_j4_labels, df_j4_preds], how = 'horizontal')
    .with_columns(
        (pl.lit("True Values").alias("Group_label")),
        (pl.lit("Predictions").alias("Group_pred")))
)

plt_j4 = \
    ggplot(df_j4)+\
    geom_line(aes(x = "DateTime", y = "Vehicles", color = "Group_label"), 
              sampling = "none", size = 0.5, show_legend = True)+\
    geom_line(aes(x = "DateTime", y = "Vehicles_preds", color = "Group_pred"), 
              sampling = "none", size = 0.5, show_legend = True)+\
    scale_color_manual(values = ['white', j4_color])+\
    scale_x_datetime(format = "%Y-%m-%d")+\
    scale_y_continuous(limits = [0, 40])+\
    theme(plot_title = element_text(hjust = 0.5),
         legend_title = element_blank())+\
    flavor_darcula()+\
    labs(x = "Date", y = "Vehicles", title = "Junction 4")

# Results
results_plts = GGBunch()
results_plts.add_plot(plt_j1, 0, 0, 700, 300)
results_plts.add_plot(plt_j2, 0, 320, 700, 300)
results_plts.add_plot(plt_j3, 0, 640, 700, 300)
results_plts.add_plot(plt_j4, 0, 960, 700, 300)
results_plts

<a id="section-seven"></a>
<div style="padding:20px; 
            color:#150d0a;
            margin:10px;
            font-size:220%;
            text-align:center;
            display:fill;
            border-radius:20px;
            border-width: 5px;
            border-style: solid;
            border-color: #150d0a;
            background-color:#63F3B9;
            overflow:hidden;
            font-weight:500">Conclusions and Potential Improvements</div>

## Conclusions and Potential Improvements:

In this notebook I created four time series models to predict one month of traffic flow at four seperate junctions. The stacked regression + XGB model tuned with Optuna showed a decent ability to capture the overall pattern of vehicle traffic, but there is still plenty of room for improvement (especially for junctions 3 and 4). 